## Fine tuning the model

This section does not have to be re-run unless we have new training data. This is for fine tuning the model

In [1]:
import openai
import os

In [2]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [15]:
response = openai.File.create(
  file=open("./Datasets/test10k.jsonl", "rb"),
  purpose='fine-tune',
  user_provided_filename="train"
)

file_id = response['id']
print(f"File successfully uploaded with ID: {file_id}")


File successfully uploaded with ID: file-83hRPnzwHCA8qWpNakrWchfC


In [16]:
#check file status
openai.FineTuningJob.list()

<OpenAIObject list at 0x11c944400> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-mYI9QyCFIcBGfWweAq7mW4FT",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1698200871,
      "finished_at": 1698201184,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:anote:8DNzdQmV",
      "organization_id": "org-WM0C8DmpfiXb4ySIeUGFLcf2",
      "result_files": [
        "file-1VwcGlPkVu9xUluWC4h6DcWM"
      ],
      "status": "succeeded",
      "validation_file": null,
      "training_file": "file-yBh58DabfZY0hUh2BqPFZamM",
      "hyperparameters": {
        "n_epochs": 4
      },
      "trained_tokens": 52492,
      "error": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-Pkd9Q8ulAtB7EbbTc6aY58Gd",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1696987687,
      "finished_at": 1696988031,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:anote:88IOfgTc",
      "organization_id": "org-WM0C8

In [17]:
response_finetuned = openai.FineTuningJob.create(
    training_file=file_id,
    model="gpt-3.5-turbo",
    suffix="anote") 

job_id = response_finetuned['id']

print(f"Fine-tuning job created successfully with ID: {job_id}")

Fine-tuning job created successfully with ID: ftjob-ClS1ABfjWqxNThtkysDDUQGF


In [21]:
# Retrieve the state of a fine-tune
retrieve_job = openai.FineTuningJob.retrieve(job_id)
print(retrieve_job)


fine_tuned_model_id = retrieve_job["fine_tuned_model"]
print(f"Fine-tuned model id: {fine_tuned_model_id}")



{
  "object": "fine_tuning.job",
  "id": "ftjob-ClS1ABfjWqxNThtkysDDUQGF",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698201395,
  "finished_at": 1698201734,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:anote:8DO8V2LB",
  "organization_id": "org-WM0C8DmpfiXb4ySIeUGFLcf2",
  "result_files": [
    "file-4AM0zYN7hzzFiPTsVskpeu62"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-83hRPnzwHCA8qWpNakrWchfC",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 53235,
  "error": null
}
Fine-tuned model id: ft:gpt-3.5-turbo-0613:personal:anote:8DO8V2LB


## Model IDs for future reference: 
- Model trained on sarcastic data has model id: ft:gpt-3.5-turbo-0613:personal:anote:85SGpKuh
- Model trained on just apple dataset has model id: ft:gpt-3.5-turbo-0613:personal:anote:88IOfgTc
- Model trained on all 3 (Apple, BofA, NVIDIA) has model id: ft:gpt-3.5-turbo-0613:personal:anote:8DO8V2LB

# Using the fine tuned model

In this section, we can use the model that has been trained on the fine tuning data

In [23]:
completion = openai.ChatCompletion.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are a factual chatbot that answers questions about 10-K documents. You only answer with answers you find in the text, no outside information."},
    {"role": "user", "content": "This is our information from the 10-K: Business Seasonality and Product IntroductionsThe Company has historically experienced higher net sales in its ﬁrst quarter compared to other quarters in its ﬁscal year due in part toseasonal holiday demand. Additionally, new product and service introductions can signiﬁcantly impact net sales, cost of sales andoperating expenses. The timing of product introductions can also impact the Company’s net sales to its indirect distribution channels asthese channels are ﬁlled with new inventory following a product launch, and channel inventory of an older product often declines as thelaunch of a newer product approaches. Net sales can also be affected when consumers and distributors anticipate a productintroduction.Human CapitalThe Company believes it has a talented, motivated and dedicated team, and works to create an inclusive, safe and supportiveenvironment for all of its team members. As of September 24, 2022, the Company had approximately 164,000 full-time equivalentemployees.Workplace Practices and PoliciesThe Company is an equal opportunity employer committed to inclusion and diversity and to providing a workplace free of harassment ordiscrimination.Compensation and BeneﬁtsThe Company believes that compensation should be competitive and equitable, and should enable employees to share in theCompany’s success. The Company recognizes its people are most likely to thrive when they have the resources to meet their needs andthe time and support to succeed in their professional and personal lives. In support of this, the Company offers a wide variety of beneﬁtsfor employees around the world and invests in tools and resources that are designed to support employees’ individual growth anddevelopment.Inclusion and DiversityThe Company remains committed to its vision to build and sustain a more inclusive workforce that is representative of the communities itserves. The Company continues to work to increase diverse representation at every level, foster an inclusive culture, and. Now, this is our question: Which quarter does Apple usually have higher net sales in?"}
  ]
)
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "The Company has historically experienced higher net sales in its \ufb01rst quarter compared to other quarters in its \ufb01scal year due in part to seasonal holiday demand"
}
